In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import re
from requests_html import HTMLSession # https://github.com/psf/requests-html

On cherche par rapport au nom de famille, les noms de famille de naissance sont utilisés. Il faut donc regarder le lieu de naissance des enfants pour savoir si la famille à bouger (suivie le/la partenaire).

Il faudrait arriver à trouver l'ancêtre le plus loin pour un nom de famille, puis analyser sa descendance :
Pour chaque enfant : 
    - Si c'est un garcon ou une fille on récupère les informations nescessaires + récupération des informations de l'épouse/époux
    - Si c'est un garcon on analysera la descendance


Informations à récupérer pour chaque personne :
 - Genre
 - Nom
 - Prénom
 - Date naissance
 - Lieu de naissance
 - Date mort
 - Lieu de mort


Visualisation (folium):
 - Sur une carte qui s'actualise tous les 10 ans, on affiche les naissances et les morts aux endroits correspondants

Avec le lieu de naissance de toute la famille, on peut voir comment ils se sont déplacé au fil des ans.

Améliorations :

Utilisation du lieu de mort, du lieu de certains événements, des informations des époux/épouses




In [ ]:
# set up 
# !pip install selenium
# !pip install bs4
# !pip install webdriver_manager
# !pip install requests


In [ ]:
# initialisation

# vidéo tuto : https://www.youtube.com/watch?v=3fcKKZMFbyA 


data_test = [{"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"},{"nom":"MARTIN", "prénom":"Jacques", "genre": "M", "ddn": 1822, "ldn":"Lyon", "ddm": 1898, "ldm":"Paris"}]

# print(data_test)
data_test = sorted(data_test, key=lambda d: d['ddn']) 
# print(data_test)


In [ ]:
# Futur fonction à utiliser (pour saisie du nom prenom et création de l'url correspondante)
def traitementURL():
    nom, prenom=getNomPrenom()
    url = "https://www.geneanet.org/fonds/individus/?go=1&nom=" + nom + "&prenom=" + prenom + "&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10" 
    return url 

def getNomPrenom():
    nom = input("Saisir nom")
    prenom = input("Saisir prénom")
    return nom, prenom

def getAncetres():
    nom, prenom = getNomPrenom()
    # récupérer f&s

    # récupérer père et mère 
    
    return

In [ ]:
# Url de recherche de test
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&prenom=fran%C3%A7ois&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10"

In [ ]:
# Récupération du code html
def getSoup(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup

In [ ]:
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&page=1&prenom=fran%C3%A7ois&prenom_operateur=or&size=10&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint="

# Récupération du code html
soup = getSoup(url)

# Récupération de la div contenant toutes les urls en lien avec la recherche
resOnly = soup.find("div", {"id":"table-resultats"})
# print(resOnly)

allArbreUrl = []
# Récupération des urls
for a in resOnly.find_all('a'):
    try:
        if (re.search("^arbres_utilisateur", a["data-id-es"])):  #^arbres_utilisateur
            allArbreUrl.append(a["href"])
    except:
        ""
print(allArbreUrl) #Comporte l'url des pages utilisateurs si se sont des arbres utilisateurs
# headline = a.text-large
# print(headline)

In [ ]:
session = HTMLSession()

In [ ]:
# Récupération des ascendants (remonter facilement à l'ancetre)
def getUrlDerAscendant(url):
    print(url)
    r = session.get(url)
    soup = BeautifulSoup(r.html.html, 'html.parser')
    # Récupération du dernier ascendants
    # print(soup)
    urlDerAscendant = url
    try: 
        arbreGene =  soup.find("td", {"id":"ancestors"})
        derAscendant = arbreGene.find("a")["href"]
        urlDerAscendant = "https://gw.geneanet.org/" + derAscendant
        urlDerAscendant = getUrlDerAscendant(urlDerAscendant)
        
    except:
        print("Pas d'autre ascendent")

    return urlDerAscendant

In [ ]:
def getEpouxEnfant(person_html):
    # Récupération époux/épouse/enfants
    person_union = person_html.find("ul", {"class":"fiche_union"})
    person_epoux = person_union.findChildren("li" , recursive=False)
    urlEpoux = []
    urlChildren = []
    data = []
    try:
        for epoux in person_epoux:
            urlEpoux.append("https://gw.geneanet.org/" + epoux.find("a")["href"])
            children  = epoux.find_all("li")
            try:
                for child in children:
                    urlChildren.append("https://gw.geneanet.org/" + child.find("a")["href"])
            except:
                print("Err Children")
    except:
        print("Err Epoux")
    
    for url in urlEpoux:
        data += getDataUrl(url)
    for url in urlChildren:
        data += getDataUrl(url)

    return data

In [ ]:
def getDataUrl(url):
    data = []
    personne = {}
    print(url)
    r = session.get(url)
    soup = BeautifulSoup(r.html.html, 'html.parser')

    
    # Récupération de l'html de l'ensemble des données d'une personne
    person_html = soup.find("div", {"id":"perso"})

    # Récupération des données concernant la personne (genre, nom, prenom)
    person_title = person_html.find("div", {"id":"person-title"})
    person_title = person_title.find("h1")

    personne["genre"] = person_title.find("img")["title"]
    
    nomprenom = person_title.find_all("a")
    personne["nom"] = nomprenom[0].text
    personne["prenom"] = nomprenom[1].text

    # Récupération des données concernant la personne (ddm, ldm, ddm, ldm)
    person_data_perso = person_html.find("ul")
    
    personne["ddn"] = ""
    # date_naissance = person_data_perso.find("li").text
    # print(date_naissance)
    # print(re.search("Né le .* -", date_naissance))

    personne["ldn"] = ""
    personne["ddm"] = ""
    personne["ldm"] = ""
    
    
    # Récupération des parents
    

    print(personne)


    
    if(personne["genre"] == "H"):
        try:
            data = getEpouxEnfant(person_html)
        except:
            print("Err Epoux/Enfant")
    data.append(personne)
    print(data)
    return data

    

In [ ]:
# Récupération du code source de chaque url correspondant à une personne
# {"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"}
urlDerAscendant = ""
# for url in allArbreUrl:
for i in range(1):
    url = "https://gw.geneanet.org/marmi1?n=martin&oc=&p=francois"
    print("url de base : " + url)
    data = []
    
    urlDerAscendant = getUrlDerAscendant(url)
    print("url dernier ascendant : " + urlDerAscendant)
    data += getDataUrl(urlDerAscendant)
    print(data)
